In [13]:
import requests

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [116]:
# Vivino 탐색을 위한 driver 생성
driver = webdriver.Chrome()
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NQABteRKW-8gtWQgEa5WAFSQnmZblliUmVqSmKOWX5Rim5JanKyWn1RpW5RYkpmXXhyfWJZalJieqlZeEh1rawgAgG8ccg%3D%3D'
driver.get(url)
time.sleep(3)

In [15]:
# 페이지 끝까지 스크롤하여 모든 데이터 로드
while True:
    # 현재 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 페이지 로드 대기
    time.sleep(1)
    
    # 페이지 살짝 위로 스크롤하여 데이터 로드될 수 있도록 조정함
    driver.execute_script("window.scrollBy(0, -100);")
    time.sleep(3)
    
    #새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

In [16]:
# 반복문 종료 후 스크롤 맨 위로 올리기
driver.execute_script("window.scrollTo(0,0);")

In [17]:
# 페이지 소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 와인 카드 데이터 가져오기 및 개수 확인
wines = soup.find_all('div', class_='wineCard__wineCard--2dj2T')
len(wines)

1971

In [118]:
# wine 데이터 추출하기
from tqdm import tqdm_notebook

data1 = []
base_url = 'https://www.vivino.com'

for wine in tqdm_notebook(wines[:500]):
    
    country = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[-1].strip()
    region = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[0].strip()
    rating = wine.find('div', class_="vivinoRating_averageValue__uDdPM").get_text().strip()
    nratings = wine.find('div', class_="vivinoRating_caption__xL84P").get_text().split()[0].strip() # rating 수 
    label = wine.find('div', class_="wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw").get_text()
    winery = wine.find('div', class_ = "wineInfoVintage__truncate--3QAtw").get_text()
    price = wine.find('div', class_="addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5").get_text().split()[-1].replace('₩', '')

    # Crawling wine varieties
    wine_link = wine.find('a',class_="anchor_anchor__m8Qi- wineCard__cardLink--3F_uB")['href']
    link = base_url + wine_link

    driver.get(link)
    time.sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    time.sleep(2)
    
    wine_html = driver.page_source
    wine_soup = BeautifulSoup(wine_html, 'html.parser')

    grape_links= wine_soup.find_all('a', class_='anchor_anchor__m8Qi-', href = True)
    grapes = [grape.text for grape in grape_links if '/grapes/' in grape['href']]
    
    # data에 값 저장하기
    data1.append({
        'Country': country,
        'Region': region,
        'Rating': rating,
        'Nratings': nratings,
        'Label': label,
        'Winery':winery,
        'Price': price,
        'Wine_Variety': grapes
    })

df1 = pd.DataFrame(data1)
data1.to_csv('./wine_df1.csv', index = False)


/var/folders/10/g_4wtq6d0k5gc_g_zkc4v8c00000gn/T/ipykernel_86359/4186595474.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wine in tqdm_notebook(wines[:500]):


  0%|          | 0/500 [00:00<?, ?it/s]

In [119]:
# wine 데이터 추출하기 (#500~)

data2 = []

for wine in tqdm_notebook(wines[500:1000]):
    
    country = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[-1].strip()
    region = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[0].strip()
    rating = wine.find('div', class_="vivinoRating_averageValue__uDdPM").get_text().strip()
    nratings = wine.find('div', class_="vivinoRating_caption__xL84P").get_text().split()[0].strip() # rating 수 
    label = wine.find('div', class_="wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw").get_text()
    winery = wine.find('div', class_ = "wineInfoVintage__truncate--3QAtw").get_text()
    price = wine.find('div', class_="addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5").get_text().split()[-1].replace('₩', '')

    # Crawling wine varieties
    wine_link = wine.find('a',class_="anchor_anchor__m8Qi- wineCard__cardLink--3F_uB")['href']
    link = base_url + wine_link

    driver.get(link)
    time.sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    time.sleep(2)
    
    wine_html = driver.page_source
    wine_soup = BeautifulSoup(wine_html, 'html.parser')

    grape_links= wine_soup.find_all('a', class_='anchor_anchor__m8Qi-', href = True)
    grapes = [grape.text for grape in grape_links if '/grapes/' in grape['href']]
    
    # data에 값 저장하기
    data2.append({
        'Country': country,
        'Region': region,
        'Rating': rating,
        'Nratings': nratings,
        'Label': label,
        'Winery':winery,
        'Price': price,
        'Wine_Variety': grapes
    })

df2 = pd.DataFrame(data2)
data2.to_csv('./wine_df2.csv', index = False)


/var/folders/10/g_4wtq6d0k5gc_g_zkc4v8c00000gn/T/ipykernel_86359/2806064068.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wine in tqdm_notebook(wines[500:1000]):


  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# wine 데이터 추출하기 (#~)

data3 = []

for wine in tqdm_notebook(wines[1000:1500]):
    
    country = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[-1].strip()
    region = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[0].strip()
    rating = wine.find('div', class_="vivinoRating_averageValue__uDdPM").get_text().strip()
    nratings = wine.find('div', class_="vivinoRating_caption__xL84P").get_text().split()[0].strip() # rating 수 
    label = wine.find('div', class_="wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw").get_text()
    winery = wine.find('div', class_ = "wineInfoVintage__truncate--3QAtw").get_text()
    price = wine.find('div', class_="addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5").get_text().split()[-1].replace('₩', '')

    # Crawling wine varieties
    wine_link = wine.find('a',class_="anchor_anchor__m8Qi- wineCard__cardLink--3F_uB")['href']
    link = base_url + wine_link

    driver.get(link)
    time.sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    time.sleep(2)
    
    wine_html = driver.page_source
    wine_soup = BeautifulSoup(wine_html, 'html.parser')

    grape_links= wine_soup.find_all('a', class_='anchor_anchor__m8Qi-', href = True)
    grapes = [grape.text for grape in grape_links if '/grapes/' in grape['href']]
    
    # data에 값 저장하기
    data.append({
        'Country': country,
        'Region': region,
        'Rating': rating,
        'Nratings': nratings,
        'Label': label,
        'Winery':winery,
        'Price': price,
        'Wine_Variety': grapes
    })

df3 = pd.DataFrame(data3)
data3.to_csv('./wine_df3.csv', index = False)

In [ ]:
data4 = []

for wine in tqdm_notebook(wines[1500:]):
    
    country = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[-1].strip()
    region = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[0].strip()
    rating = wine.find('div', class_="vivinoRating_averageValue__uDdPM").get_text().strip()
    nratings = wine.find('div', class_="vivinoRating_caption__xL84P").get_text().split()[0].strip() # rating 수 
    label = wine.find('div', class_="wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw").get_text()
    winery = wine.find('div', class_ = "wineInfoVintage__truncate--3QAtw").get_text()
    price = wine.find('div', class_="addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5").get_text().split()[-1].replace('₩', '')

    # Crawling wine varieties
    wine_link = wine.find('a',class_="anchor_anchor__m8Qi- wineCard__cardLink--3F_uB")['href']
    link = base_url + wine_link

    driver.get(link)
    time.sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    time.sleep(2)
    
    wine_html = driver.page_source
    wine_soup = BeautifulSoup(wine_html, 'html.parser')

    grape_links= wine_soup.find_all('a', class_='anchor_anchor__m8Qi-', href = True)
    grapes = [grape.text for grape in grape_links if '/grapes/' in grape['href']]
    
    # data에 값 저장하기
    data.append({
        'Country': country,
        'Region': region,
        'Rating': rating,
        'Nratings': nratings,
        'Label': label,
        'Winery':winery,
        'Price': price,
        'Wine_Variety': grapes
    })

df4 = pd.DataFrame(data4)
data4.to_csv('./wine_df4.csv', index = False)

In [ ]:
driver.quit()